# Leitura do arquivo

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog


# Representa a janela principal
janela = tk.Tk()

# Esconde a GUI e mostra somente o pop-up da próxima função
janela.withdraw()

# Abre o pop-up de seleção de arquivo com a função askopenfilename()
filePathIn = filedialog.askopenfilename(filetypes=[("arquivos CSV", "*.csv")])

try:
    # Encoding usado para ler CSV separado por ponto e vírgula
    dfGiss = pd.read_csv(filePathIn, encoding='latin1', sep=';', usecols=[14, 25, 24, 6, 7, 1, 19, 4, 3, 23])

    # Substitui ponto por vírgula
    dfGiss.iloc[:, 0:3] = dfGiss.iloc[:, 0:3].astype(str).apply(lambda x: x.str.replace('.', ','))
    dfGiss.iloc[:, 6] = dfGiss.iloc[:, 6].astype(str).apply(lambda x: x.replace('.', ''))

    # Edita as colunas
    dia = dfGiss['DIA'].astype(str).str.zfill(2)
    mes = dfGiss['MES_NUM'].astype(str).str.zfill(2)
    dfGiss['TOMA_CPF_CGC'] = dfGiss['TOMA_CPF_CGC'].astype(str).str.zfill(14)
    dfGiss['ATIT_COD_ATIVIDADE'] = dfGiss['ATIT_COD_ATIVIDADE'].str.zfill(16).str.slice(0, 4)
    ano = dfGiss['ANO'].astype(str)
    dfGiss['DIA'] = dia + '/' + mes + '/' + ano + ' 00:00:00'

    # Renomeia as colunas
    dfGiss.rename(columns={'ENFS_NUM_NFS_INI': 'Nº NFS-e', 'DIA': 'Data Hora NFE',
                           'TOMA_CPF_CGC': 'CPF/CNPJ do Prestador', 'VALOR_FATURADO': 'Valor dos Serviços',
                           'ATIT_COD_ATIVIDADE': 'Código do Serviço Prestado na Nota Fiscal',
                           'VALOR_ALIQUOTA': 'Alíquota', 'VALOR_IMPOSTO': 'ISS devido',
                           'TOMA_STA_ESTABELECIDO': 'ISS Retido'}, inplace=True)

    # Adiciona colunas vazias
    colunasG5 = ['Código de Verificação da NFS-e', 'Tipo de RPS', 'Série do RPS', 'Número do RPS',
                 'Inscrição Municipal do Prestador', 'Tipo do Endereço do Prestador', 'Endereço do Prestador',
                 'Número do Endereço do Prestador', 'Complemento do Endereço do Prestador', 'Bairro do Prestador',
                 'Cidade do Prestador', 'Razão Social do Prestador', 'UF do Prestador', 'CEP do Prestador',
                 'Email do Prestador', 'Data de Cancelamento', 'Campo Reservado', 'Nº da Guia',
                 'Data de Quitação da Guia Vinculada a Nota Fiscal', 'Inscrição Municipal do Tomador',
                 'Inscrição Estadual do Tomador', 'Razão Social do Tomador', 'Tipo do Endereço do Tomador',
                 'Nº NFS-e Consolidada', 'Endereço do Tomador', 'Número do Endereço do Tomador',
                 'Complemento do Endereço do Tomador', 'Bairro do Tomador', 'CEP do Tomador', 'Email do Tomador',
                 'Nº NFS-e Substituta', 'ISS recolhido', 'CPF/CNPJ do Intermediário',
                 'Inscrição Municipal do Intermediário', 'Razão Social do Intermediário', 'Repasse do Plano de Saúde',
                 'Carga tributária: Porcentagem', 'Carga tributária: Fonte', 'Situação do Aceite',
                 'Tipo de Consolidação', 'Discriminação dos Serviços']
    for colunasVazias in colunasG5:
        dfGiss[colunasVazias] = ''
        # Adiciona colunas com valores específicos
    cnpjTomador = simpledialog.askstring("CNPJ", "Digite o CNPJ do Cliente")
    if cnpjTomador is None:
        messagebox.showinfo("Cancelar", "Operação cancelada pelo usuário.")
        exit()
    nomeCidade = simpledialog.askstring("CIDADE", "Digite o municipio do Cliente")
    if nomeCidade is None:
        messagebox.showinfo("Cancelar", "Operação cancelada pelo usuário.")
        exit()
        
    dfGiss['Data do Fato Gerador'] = dfGiss['Data Hora NFE'].str.slice(0, 10)
    dfGiss['Tipo de Registro'] = ['2'] * len(dfGiss)
    dfGiss['UF do Tomador'] = ['SP'] * len(dfGiss)
    dfGiss = dfGiss.assign(**{'Indicador de CPF/CNPJ do Prestador': '2'})
    dfGiss = dfGiss.assign(**{'Situação da Nota Fiscal': 'T'})
    dfGiss = dfGiss.assign(**{'Indicador de CPF/CNPJ do Tomador': '2'})
    dfGiss = dfGiss.assign(**{'CPF/CNPJ do Tomador': cnpjTomador})
    dfGiss = dfGiss.assign(**{'Cidade do Tomador': nomeCidade})
    dfGiss = dfGiss.assign(**{'Indicador de CPF/CNPJ do Intermediário': '3'})

    # Adiciona colunas com valor 0
    colunazero = ['Opção Pelo Simples', 'Valor das Deduções', 'Valor do Crédito', 'ISS a recolher', 'PIS/PASEP',
                  'COFINS', 'INSS', 'IR', 'CSLL', 'Carga tributária: Valor', 'CEI', 'Matrícula da Obra',
                  'Município Prestação - cód. IBGE', 'Encapsulamento', 'Valor Total Recebido']
    for coluna in colunazero:
        dfGiss[coluna] = '0'

    # Ordem das colunas
    ordemColunas = ["Tipo de Registro", "Nº NFS-e", "Data Hora NFE", "Código de Verificação da NFS-e", "Tipo de RPS",
                    "Série do RPS", "Número do RPS", "Data do Fato Gerador", "Inscrição Municipal do Prestador",
                    "Indicador de CPF/CNPJ do Prestador", "CPF/CNPJ do Prestador", "Razão Social do Prestador",
                    "Tipo do Endereço do Prestador", "Endereço do Prestador", "Número do Endereço do Prestador",
                    "Complemento do Endereço do Prestador", "Bairro do Prestador", "Cidade do Prestador",
                    "UF do Prestador", "CEP do Prestador", "Email do Prestador", "Opção Pelo Simples",
                    "Situação da Nota Fiscal", "Data de Cancelamento", "Nº da Guia",
                    "Data de Quitação da Guia Vinculada a Nota Fiscal", "Valor dos Serviços", "Valor das Deduções",
                    "Código do Serviço Prestado na Nota Fiscal", "Alíquota", "ISS devido", "Valor do Crédito",
                    "ISS Retido", "Indicador de CPF/CNPJ do Tomador", "CPF/CNPJ do Tomador",
                    "Inscrição Municipal do Tomador", "Inscrição Estadual do Tomador", "Razão Social do Tomador",
                    "Tipo do Endereço do Tomador", "Endereço do Tomador", "Número do Endereço do Tomador",
                    "Complemento do Endereço do Tomador", "Bairro do Tomador", "Cidade do Tomador", "UF do Tomador",
                    "CEP do Tomador", "Email do Tomador", "Nº NFS-e Substituta", "ISS recolhido", "ISS a recolher",
                    "Indicador de CPF/CNPJ do Intermediário", "CPF/CNPJ do Intermediário",
                    "Inscrição Municipal do Intermediário", "Razão Social do Intermediário",
                    "Repasse do Plano de Saúde", "PIS/PASEP", "COFINS", "INSS", "IR", "CSLL",
                    "Carga tributária: Valor", "Carga tributária: Porcentagem", "Carga tributária: Fonte", "CEI",
                    "Matrícula da Obra", "Município Prestação - cód. IBGE", "Situação do Aceite", "Encapsulamento",
                    "Valor Total Recebido", "Tipo de Consolidação", "Nº NFS-e Consolidada", "Campo Reservado",
                    'Discriminação dos Serviços']
    dfGiss = dfGiss[ordemColunas]

    # Pergunta ao usuário se ele deseja salvar o arquivo
    if messagebox.askyesno("Salvar Arquivo", "Deseja salvar o arquivo?"):
        # Abre o pop-up de seleção de caminho para salvar o arquivo
        file_path_out = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("Arquivos CSV", "*.csv")])
                                                     
        # Salva o DataFrame como um arquivo CSV
        dfGiss.to_csv(file_path_out, sep=';', index=False)
        messagebox.showinfo("Sucesso", "Arquivo salvo com sucesso!")
    else:
        messagebox.showinfo("Cancelar", "Operação cancelada pelo usuário.")

except Exception as error:
    print("Ocorreu o seguinte erro:", error)



###### Nesse código, apply(lambda x: x.str.replace('.',',')) é usado para aplicar a função str.replace() em todas as células das primeiras três colunas. Isso garante que a substituição seja feita corretamente, independentemente do tipo de dados original das colunas.